# <a href="https://colab.research.google.com/github/forrestshort/electrodynamics/blob/master/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Midterm Project

Forrest Short

## Helmholtz Coil

This project is a visualization of the magnetic field produced by a Helmholtz coil



## Load Packages


In [78]:
try:
    # Import gmsh library for generating meshes.
    import gmsh
except ImportError:
    # If it is not available, install it.  Then import it.
    !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
    import gmsh
    
try:
    # Import FEniCSx libraries for finite element analysis.
    import dolfinx
except ImportError:
    # If they are not found, install them.  Then import them.
    !wget "https://fem-on-colab.github.io/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx
    
try:
    # Import multiphenicsx, mainly for plotting.
    import multiphenicsx
except ImportError:
    # If they are not found, install them.
    !pip3 install "multiphenicsx@git+https://github.com/multiphenics/multiphenicsx.git@8b97b4e"
    import multiphenicsx
    
import dolfinx.fem
import dolfinx.io
import gmsh
import mpi4py.MPI
import numpy as np
import petsc4py.PETSc
import ufl
from dolfinx.io import gmshio
#import multiphenicsx.fem
import multiphenicsx.io

## Simulation Inputs


In [79]:
# Mesh Dimension
dim = 2

# Radius of background disk
r_background = 5

# µ background 
mu_background = 1

# µ wires
mu_wire = 6

# Radius of Coils
r_wire = 0.1
r_helm = 0.12

# Number of Coils.
N = 4

# Current density in each coil.
J0 = 5

## Helmholtz Coil

In [80]:
# Create a model.
gmsh.initialize()
gmsh.model.add("mesh")

inner_tag = 2
outer_tag = 2 + N
background_surfaces = []
other_surfaces = []


# Define the system: a large disk.
background = gmsh.model.occ.addDisk(0, 0, 0, r_background, r_background)
gmsh.model.occ.synchronize()


wires_in = []
wires_out = []

wires_out.append( (2, gmsh.model.occ.addDisk( 1, -0.5, 0, r_wire, r_wire)) )
wires_out.append( (2, gmsh.model.occ.addDisk( 1,  0.5, 0, r_wire, r_wire)) )

wires_in.append( (2, gmsh.model.occ.addDisk(-1, -0.5, 0, r_wire, r_wire)) )
wires_in.append( (2, gmsh.model.occ.addDisk(-1,  0.5, 0, r_wire, r_wire)) )

# Update the model.
gmsh.model.occ.synchronize()

# Resolve the boundaries of the wires and ring in the background domain.

all_surfaces = []
all_surfaces.extend(wires_in)
all_surfaces.extend(wires_out)
whole_domain = gmsh.model.occ.fragment([(2, background)], all_surfaces)

gmsh.model.occ.synchronize()



for domain in whole_domain[0]:
    center = gmsh.model.occ.getCenterOfMass(domain[0], domain[1])
    mass = gmsh.model.occ.getMass(domain[0], domain[1])

    
    # Identify the background circle by its center of mass
    if np.allclose(center, [0, 0, 0]):
        background_surfaces.append(domain[1])

    # Identify the inner wires by their centers of mass.
    elif np.isclose(center[0], -1):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], inner_tag)
        inner_tag +=1
        other_surfaces.append(domain)


    # Identify the outer wires by their center of mass.
    elif np.isclose(center[0], +1):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], outer_tag)
        outer_tag +=1
        other_surfaces.append(domain)


# Add marker for the vacuum.
gmsh.model.addPhysicalGroup(2, background_surfaces, tag=0)

# Create mesh resolution that is fine around the wires and
# make the grid coarse further away from the ring.
gmsh.model.mesh.field.add("Distance", 1)
edges = gmsh.model.getBoundary(other_surfaces, oriented=False)
gmsh.model.mesh.field.setNumbers(1, "EdgesList", [e[1] for e in edges])
gmsh.model.mesh.field.add("Threshold", 2)
gmsh.model.mesh.field.setNumber(2, "IField", 1)
gmsh.model.mesh.field.setNumber(2, "LcMin", r_wire / 2)
gmsh.model.mesh.field.setNumber(2, "LcMax", 5 * r_wire)
gmsh.model.mesh.field.setNumber(2, "DistMin", 2 * r_wire)
gmsh.model.mesh.field.setNumber(2, "DistMax", 4 * r_wire)
gmsh.model.mesh.field.setAsBackgroundMesh(2)
gmsh.option.setNumber("Mesh.Algorithm", 7)

# Create a mesh for this system.
gmsh.model.mesh.generate(dim)

# Bring the mesh into FEniCSx.
mesh, subdomains, boundaries = dolfinx.io.gmshio.model_to_mesh(
    gmsh.model, comm=mpi4py.MPI.COMM_WORLD, rank=0, gdim=2)

gmsh.finalize()

# This loop will assign material properties to each cell in our model.
# In this case, it is the relative magnetic permeability and current density.

# Define a simple function space for properties.
Q = dolfinx.fem.FunctionSpace(mesh, ("DG", 0))

# Get the list of materials.
material_tags = np.unique(subdomains.values)

# Define functions for current density and magnetic permeability.
mu = dolfinx.fem.Function(Q)
J = dolfinx.fem.Function(Q)

# Only some regions carry current. Initialize all current densities to zero.
J.x.array[:] = 0.0

# Now, cycle over all objects and assign material properties. 
for tag in material_tags:
    cells = subdomains.find(tag)
    
    # Set values for magnetic permeability.
    if tag == 0:
        # Vacuum
        mu_ = mu_background
    elif tag == 1:
        # Ring
        mu_ = mu_ring
    else:
        # Wire
        mu_ = mu_wire

    mu.x.array[cells] = np.full_like(cells, mu_, dtype=petsc4py.PETSc.ScalarType)
    
    # Set nonzero current densities.
    if tag in range(2, 2+N):
        J.x.array[cells] = np.full_like(cells, J0, dtype=petsc4py.PETSc.ScalarType)
    elif tag in range(2+N, 2*N + 2):
        J.x.array[cells] = np.full_like(cells, -J0, dtype=petsc4py.PETSc.ScalarType)
        
## Set up the finite element problem.

# Define trial and test functions.
V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 2))

# Define the trial and test functions.
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Create a function to store the solution.
# This is the vector potential.  A_x = A_y = 0.
A_z = dolfinx.fem.Function(V)

# Identify the domain and boundary.
D = mesh.topology.dim
Omega = dolfinx.mesh.locate_entities_boundary(mesh, D-1, lambda x: np.full(x.shape[1], True))
dOmega = dolfinx.fem.locate_dofs_topological(V, D-1, Omega)

# Force the potential to vanish on the boundary.
bc = dolfinx.fem.dirichletbc(petsc4py.PETSc.ScalarType(0), dOmega, V)

# Define the Poisson equation we are trying to solve.
a = (1 / mu) * ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = 4 * ufl.pi * J * v * ufl.dx

# Define the problem.
problem = dolfinx.fem.petsc.LinearProblem(a, L, u=A_z, bcs=[bc])

# Solve the problem.
problem.solve()


# Compute the magnetic field.
W = dolfinx.fem.VectorFunctionSpace(mesh, ("CG", 2))
B = dolfinx.fem.Function(W)
B_expr = dolfinx.fem.Expression(ufl.as_vector((A_z.dx(1), -A_z.dx(0))), W.element.interpolation_points())
B.interpolate(B_expr)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 20%] Meshing curve 2 (Ellipse)
Info    : [ 40%] Meshing curve 3 (Ellipse)
Info    : [ 60%] Meshing curve 4 (Ellipse)
Info    : [ 80%] Meshing curve 5 (Ellipse)
Info    : Done meshing 1D (Wall 0.00140558s, CPU 0.002107s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 2 (Plane, Bamg)
Info    : [  0%] BAMG succeeded 22 vertices 29 triangles
Info    : [  0%] BAMG succeeded 22 vertices 29 triangles
Info    : [ 20%] Meshing surface 3 (Plane, Bamg)
Info    : [ 20%] BAMG succeeded 22 vertices 29 triangles
Info    : [ 20%] BAMG succeeded 22 vertices 29 triangles
Info    : [ 40%] Meshing surface 4 (Plane, Bamg)
Info    : [ 40%] BAMG succeeded 21 vertices 27 triangles
Info    : [ 40%] BAMG succeeded 21 vertices 27 triangles
Info    : [ 60%] Meshing surface 5 (Plane, Bamg)
Info    : [ 60%] BAMG succeeded 22 vertices 29 triangles
Info    : [ 60%] BAMG succeeded 22 vertices 29 triangles
Info    : [ 80%] Meshing 

In [81]:
# Helmholtz
multiphenicsx.io.plot_scalar_field(A_z,"Vector Potential", warp_factor=1)


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [82]:
multiphenicsx.io.plot_vector_field(B,"Magnetic Field", glyph_factor=0.1)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## 3 Coils

In [83]:
# Create a model.
gmsh.initialize()
gmsh.model.add("mesh")

# Define the system: a large disk.
background = gmsh.model.occ.addDisk(0, 0, 0, r_background, r_background)
gmsh.model.occ.synchronize()


wires_in = []
wires_out = []

wires_out.append( (2, gmsh.model.occ.addDisk( 1, -0.5, 0, r_wire, r_wire)) )
wires_out.append( (2, gmsh.model.occ.addDisk( 1,  0, 0, r_helm, r_helm)) )
wires_out.append( (2, gmsh.model.occ.addDisk( 1,  0.5, 0, r_wire, r_wire)) )

wires_in.append( (2, gmsh.model.occ.addDisk(-1, -0.5, 0, r_wire, r_wire)) )
wires_in.append( (2, gmsh.model.occ.addDisk(-1,  0, 0, r_helm, r_helm)) )
wires_in.append( (2, gmsh.model.occ.addDisk(-1,  0.5, 0, r_wire, r_wire)) )

# Update the model.
gmsh.model.occ.synchronize()

# Resolve the boundaries of the wires and ring in the background domain.

all_surfaces = []
all_surfaces.extend(wires_in)
all_surfaces.extend(wires_out)
whole_domain = gmsh.model.occ.fragment([(2, background)], all_surfaces)

# Update the model.
gmsh.model.occ.synchronize()

# Create physical markers for each object.
# Use the following markers:
# - Vacuum: 0
# - Ring: 1
# - Inner wires: $[2,3,\dots,N+1]$
# - Outer wires: $[N+2,\dots, 2\cdot N+1]$
inner_tag = 2
outer_tag = 2 + N
background_surfaces = []
other_surfaces = []

# Gmsh can compute the mass of objects and the location of their
# centers of mass.  This loop uses these properties to determine
# which object to associate grid points with.
# 
# We will use these tags to define material properties later.
for domain in whole_domain[0]:
    center = gmsh.model.occ.getCenterOfMass(domain[0], domain[1])
    mass = gmsh.model.occ.getMass(domain[0], domain[1])

    
    # Identify the background circle by its center of mass
    if np.allclose(center, [0, 0, 0]):
        background_surfaces.append(domain[1])

    # Identify the inner wires by their centers of mass.
    elif np.isclose(center[0], -1):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], inner_tag)
        inner_tag +=1
        other_surfaces.append(domain)

    elif np.isclose(center[0], -1.2):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], inner_tag)
        inner_tag +=1
        other_surfaces.append(domain)

    # Identify the outer wires by their center of mass.
    elif np.isclose(center[0], +1):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], outer_tag)
        outer_tag +=1
        other_surfaces.append(domain)

    elif np.isclose(center[0], +1.2):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], outer_tag)
        outer_tag +=1
        other_surfaces.append(domain)

# Add marker for the vacuum.
gmsh.model.addPhysicalGroup(2, background_surfaces, tag=0)

# Create mesh resolution that is fine around the wires and
# make the grid coarse further away from the ring.
gmsh.model.mesh.field.add("Distance", 1)
edges = gmsh.model.getBoundary(other_surfaces, oriented=False)
gmsh.model.mesh.field.setNumbers(1, "EdgesList", [e[1] for e in edges])
gmsh.model.mesh.field.add("Threshold", 2)
gmsh.model.mesh.field.setNumber(2, "IField", 1)
gmsh.model.mesh.field.setNumber(2, "LcMin", r_wire / 2)
gmsh.model.mesh.field.setNumber(2, "LcMax", 5 * r_wire)
gmsh.model.mesh.field.setNumber(2, "DistMin", 2 * r_wire)
gmsh.model.mesh.field.setNumber(2, "DistMax", 4 * r_wire)
gmsh.model.mesh.field.setAsBackgroundMesh(2)
gmsh.option.setNumber("Mesh.Algorithm", 7)

# Create a mesh for this system.
gmsh.model.mesh.generate(dim)

# Bring the mesh into FEniCSx.
mesh, subdomains, boundaries = dolfinx.io.gmshio.model_to_mesh(
    gmsh.model, comm=mpi4py.MPI.COMM_WORLD, rank=0, gdim=2)

gmsh.finalize()

# This loop will assign material properties to each cell in our model.
# In this case, it is the relative magnetic permeability and current density.

# Define a simple function space for properties.
Q = dolfinx.fem.FunctionSpace(mesh, ("DG", 0))

# Get the list of materials.
material_tags = np.unique(subdomains.values)

# Define functions for current density and magnetic permeability.
mu = dolfinx.fem.Function(Q)
J = dolfinx.fem.Function(Q)

# Only some regions carry current. Initialize all current densities to zero.
J.x.array[:] = 0.0

# Now, cycle over all objects and assign material properties. 
for tag in material_tags:
    cells = subdomains.find(tag)
    
    # Set values for magnetic permeability.
    if tag == 0:
        # Vacuum
        mu_ = mu_background
    elif tag == 1:
        # Ring
        mu_ = mu_ring
    else:
        # Wire
        mu_ = mu_wire

    mu.x.array[cells] = np.full_like(cells, mu_, dtype=petsc4py.PETSc.ScalarType)
    
    # Set nonzero current densities.
    if tag in range(2, 2+N):
        J.x.array[cells] = np.full_like(cells, J0, dtype=petsc4py.PETSc.ScalarType)
    elif tag in range(2+N, 2*N + 2):
        J.x.array[cells] = np.full_like(cells, -J0, dtype=petsc4py.PETSc.ScalarType)
        
## Set up the finite element problem.

# Define trial and test functions.
V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 2))

# Define the trial and test functions.
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Create a function to store the solution.
# This is the vector potential.  A_x = A_y = 0.
A_z = dolfinx.fem.Function(V)

# Identify the domain and boundary.
D = mesh.topology.dim
Omega = dolfinx.mesh.locate_entities_boundary(mesh, D-1, lambda x: np.full(x.shape[1], True))
dOmega = dolfinx.fem.locate_dofs_topological(V, D-1, Omega)

# Force the potential to vanish on the boundary.
bc = dolfinx.fem.dirichletbc(petsc4py.PETSc.ScalarType(0), dOmega, V)

# Define the Poisson equation we are trying to solve.
a = (1 / mu) * ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = 4 * ufl.pi * J * v * ufl.dx

# Define the problem.
problem = dolfinx.fem.petsc.LinearProblem(a, L, u=A_z, bcs=[bc])

# Solve the problem.
problem.solve()


# Compute the magnetic field.
W = dolfinx.fem.VectorFunctionSpace(mesh, ("CG", 2))
B = dolfinx.fem.Function(W)
B_expr = dolfinx.fem.Expression(ufl.as_vector((A_z.dx(1), -A_z.dx(0))), W.element.interpolation_points())
B.interpolate(B_expr)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 20%] Meshing curve 2 (Ellipse)
Info    : [ 30%] Meshing curve 3 (Ellipse)
Info    : [ 50%] Meshing curve 4 (Ellipse)
Info    : [ 60%] Meshing curve 5 (Ellipse)
Info    : [ 80%] Meshing curve 6 (Ellipse)
Info    : [ 90%] Meshing curve 7 (Ellipse)
Info    : Done meshing 1D (Wall 0.00121213s, CPU 0.001356s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 2 (Plane, Bamg)
Info    : [  0%] BAMG succeeded 22 vertices 29 triangles
Info    : [  0%] BAMG succeeded 22 vertices 29 triangles
Info    : [ 20%] Meshing surface 3 (Plane, Bamg)
Info    : [ 20%] BAMG succeeded 31 vertices 44 triangles
Info    : [ 20%] BAMG succeeded 31 vertices 44 triangles
Info    : [ 30%] Meshing surface 4 (Plane, Bamg)
Info    : [ 30%] BAMG succeeded 22 vertices 29 triangles
Info    : [ 30%] BAMG succeeded 22 vertices 29 triangles
Info    : [ 50%] Meshing surface 5 (Plane, Bamg)
Info    : [ 50%] BAMG succeeded 22 vertices 29 triang

In [84]:
# 3 Coils
multiphenicsx.io.plot_scalar_field(A_z,"Vector Potential", warp_factor=1)


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [85]:
multiphenicsx.io.plot_vector_field(B,"Magnetic Field", glyph_factor=0.1)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Maxwell Coil

In [86]:
# Create a model.
gmsh.initialize()
gmsh.model.add("mesh")

# Define the system: a large disk.
background = gmsh.model.occ.addDisk(0, 0, 0, r_background, r_background)
gmsh.model.occ.synchronize()


wires_in = []
wires_out = []

wires_out.append( (2, gmsh.model.occ.addDisk( 1, -0.5, 0, r_wire, r_wire)) )
wires_out.append( (2, gmsh.model.occ.addDisk( 1.2,  0, 0, r_helm, r_helm)) )
wires_out.append( (2, gmsh.model.occ.addDisk( 1,  0.5, 0, r_wire, r_wire)) )

wires_in.append( (2, gmsh.model.occ.addDisk(-1, -0.5, 0, r_wire, r_wire)) )
wires_in.append( (2, gmsh.model.occ.addDisk(-1.2,  0, 0, r_helm, r_helm)) )
wires_in.append( (2, gmsh.model.occ.addDisk(-1,  0.5, 0, r_wire, r_wire)) )

# Update the model.
gmsh.model.occ.synchronize()

# Resolve the boundaries of the wires and ring in the background domain.

all_surfaces = []
all_surfaces.extend(wires_in)
all_surfaces.extend(wires_out)
whole_domain = gmsh.model.occ.fragment([(2, background)], all_surfaces)

# Update the model.
gmsh.model.occ.synchronize()

# Create physical markers for each object.
# Use the following markers:
# - Vacuum: 0
# - Ring: 1
# - Inner wires: $[2,3,\dots,N+1]$
# - Outer wires: $[N+2,\dots, 2\cdot N+1]$
inner_tag = 2
outer_tag = 2 + N
background_surfaces = []
other_surfaces = []

# Gmsh can compute the mass of objects and the location of their
# centers of mass.  This loop uses these properties to determine
# which object to associate grid points with.
# 
# We will use these tags to define material properties later.
for domain in whole_domain[0]:
    center = gmsh.model.occ.getCenterOfMass(domain[0], domain[1])
    mass = gmsh.model.occ.getMass(domain[0], domain[1])

    
    # Identify the background circle by its center of mass
    if np.allclose(center, [0, 0, 0]):
        background_surfaces.append(domain[1])

    # Identify the inner wires by their centers of mass.
    elif np.isclose(center[0], -1):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], inner_tag)
        inner_tag +=1
        other_surfaces.append(domain)

    elif np.isclose(center[0], -1.2):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], inner_tag)
        inner_tag +=1
        other_surfaces.append(domain)

    # Identify the outer wires by their center of mass.
    elif np.isclose(center[0], +1):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], outer_tag)
        outer_tag +=1
        other_surfaces.append(domain)

    elif np.isclose(center[0], +1.2):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], outer_tag)
        outer_tag +=1
        other_surfaces.append(domain)

# Add marker for the vacuum.
gmsh.model.addPhysicalGroup(2, background_surfaces, tag=0)

# Create mesh resolution that is fine around the wires and
# make the grid coarse further away from the ring.
gmsh.model.mesh.field.add("Distance", 1)
edges = gmsh.model.getBoundary(other_surfaces, oriented=False)
gmsh.model.mesh.field.setNumbers(1, "EdgesList", [e[1] for e in edges])
gmsh.model.mesh.field.add("Threshold", 2)
gmsh.model.mesh.field.setNumber(2, "IField", 1)
gmsh.model.mesh.field.setNumber(2, "LcMin", r_wire / 2)
gmsh.model.mesh.field.setNumber(2, "LcMax", 5 * r_wire)
gmsh.model.mesh.field.setNumber(2, "DistMin", 2 * r_wire)
gmsh.model.mesh.field.setNumber(2, "DistMax", 4 * r_wire)
gmsh.model.mesh.field.setAsBackgroundMesh(2)
gmsh.option.setNumber("Mesh.Algorithm", 7)

# Create a mesh for this system.
gmsh.model.mesh.generate(dim)

# Bring the mesh into FEniCSx.
mesh, subdomains, boundaries = dolfinx.io.gmshio.model_to_mesh(
    gmsh.model, comm=mpi4py.MPI.COMM_WORLD, rank=0, gdim=2)

gmsh.finalize()

# This loop will assign material properties to each cell in our model.
# In this case, it is the relative magnetic permeability and current density.

# Define a simple function space for properties.
Q = dolfinx.fem.FunctionSpace(mesh, ("DG", 0))

# Get the list of materials.
material_tags = np.unique(subdomains.values)

# Define functions for current density and magnetic permeability.
mu = dolfinx.fem.Function(Q)
J = dolfinx.fem.Function(Q)

# Only some regions carry current. Initialize all current densities to zero.
J.x.array[:] = 0.0

# Now, cycle over all objects and assign material properties. 
for tag in material_tags:
    cells = subdomains.find(tag)
    
    # Set values for magnetic permeability.
    if tag == 0:
        # Vacuum
        mu_ = mu_background
    elif tag == 1:
        # Ring
        mu_ = mu_ring
    else:
        # Wire
        mu_ = mu_wire

    mu.x.array[cells] = np.full_like(cells, mu_, dtype=petsc4py.PETSc.ScalarType)
    
    # Set nonzero current densities.
    if tag in range(2, 2+N):
        J.x.array[cells] = np.full_like(cells, J0, dtype=petsc4py.PETSc.ScalarType)
    elif tag in range(2+N, 2*N + 2):
        J.x.array[cells] = np.full_like(cells, -J0, dtype=petsc4py.PETSc.ScalarType)
        
## Set up the finite element problem.

# Define trial and test functions.
V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 2))

# Define the trial and test functions.
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Create a function to store the solution.
# This is the vector potential.  A_x = A_y = 0.
A_z = dolfinx.fem.Function(V)

# Identify the domain and boundary.
D = mesh.topology.dim
Omega = dolfinx.mesh.locate_entities_boundary(mesh, D-1, lambda x: np.full(x.shape[1], True))
dOmega = dolfinx.fem.locate_dofs_topological(V, D-1, Omega)

# Force the potential to vanish on the boundary.
bc = dolfinx.fem.dirichletbc(petsc4py.PETSc.ScalarType(0), dOmega, V)

# Define the Poisson equation we are trying to solve.
a = (1 / mu) * ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = 4 * ufl.pi * J * v * ufl.dx

# Define the problem.
problem = dolfinx.fem.petsc.LinearProblem(a, L, u=A_z, bcs=[bc])

# Solve the problem.
problem.solve()


# Compute the magnetic field.
W = dolfinx.fem.VectorFunctionSpace(mesh, ("CG", 2))
B = dolfinx.fem.Function(W)
B_expr = dolfinx.fem.Expression(ufl.as_vector((A_z.dx(1), -A_z.dx(0))), W.element.interpolation_points())
B.interpolate(B_expr)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 20%] Meshing curve 2 (Ellipse)
Info    : [ 30%] Meshing curve 3 (Ellipse)
Info    : [ 50%] Meshing curve 4 (Ellipse)
Info    : [ 60%] Meshing curve 5 (Ellipse)
Info    : [ 80%] Meshing curve 6 (Ellipse)
Info    : [ 90%] Meshing curve 7 (Ellipse)
Info    : Done meshing 1D (Wall 0.00195892s, CPU 0.002484s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 2 (Plane, Bamg)
Info    : [  0%] BAMG succeeded 22 vertices 29 triangles
Info    : [  0%] BAMG succeeded 22 vertices 29 triangles
Info    : [ 20%] Meshing surface 3 (Plane, Bamg)
Info    : [ 20%] BAMG succeeded 34 vertices 50 triangles
Info    : [ 20%] BAMG succeeded 32 vertices 46 triangles
Info    : [ 20%] BAMG succeeded 32 vertices 46 triangles
Info    : [ 30%] Meshing surface 4 (Plane, Bamg)
Info    : [ 30%] BAMG succeeded 22 vertices 29 triangles
Info    : [ 30%] BAMG succeeded 22 vertices 29 triangles
Info    : [ 50%] Meshing surface 5 (Plane, Ba

In [87]:
# Maxwell
multiphenicsx.io.plot_scalar_field(A_z,"Vector Potential", warp_factor=1)


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [74]:
multiphenicsx.io.plot_vector_field(B,"Magnetic Field", glyph_factor=0.1)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [76]:
# Create a model.
gmsh.initialize()
gmsh.model.add("mesh")

# Define the system: a large disk.
background = gmsh.model.occ.addDisk(0, 0, 0, r_background, r_background)
gmsh.model.occ.synchronize()


wires_in = []
wires_out = []

wires_out.append( (2, gmsh.model.occ.addDisk( 1, -0.5, 0, r_wire, r_wire)) )
wires_out.append( (2, gmsh.model.occ.addDisk( 1,  0.5, 0, r_wire, r_wire)) )

wires_in.append( (2, gmsh.model.occ.addDisk(-1, -0.5, 0, r_wire, r_wire)) )
wires_in.append( (2, gmsh.model.occ.addDisk(-1,  0.5, 0, r_wire, r_wire)) )

wires_out.append( (2, gmsh.model.occ.addDisk( 1,  -2, 0, r_wire, r_wire)) )
wires_out.append( (2, gmsh.model.occ.addDisk( 1,  2, 0, r_wire, r_wire)) )

wires_in.append( (2, gmsh.model.occ.addDisk(-1,  -2, 0, r_wire, r_wire)) )
wires_in.append( (2, gmsh.model.occ.addDisk(-1,  2, 0, r_wire, r_wire)) )

# Update the model.
gmsh.model.occ.synchronize()

# Resolve the boundaries of the wires and ring in the background domain.

all_surfaces = []
all_surfaces.extend(wires_in)
all_surfaces.extend(wires_out)
whole_domain = gmsh.model.occ.fragment([(2, background)], all_surfaces)

# Update the model.
gmsh.model.occ.synchronize()

# Create physical markers for each object.
# Use the following markers:
# - Vacuum: 0
# - Ring: 1
# - Inner wires: $[2,3,\dots,N+1]$
# - Outer wires: $[N+2,\dots, 2\cdot N+1]$
inner_tag = 2
outer_tag = 2 + N
background_surfaces = []
other_surfaces = []

# Gmsh can compute the mass of objects and the location of their
# centers of mass.  This loop uses these properties to determine
# which object to associate grid points with.
# 
# We will use these tags to define material properties later.
for domain in whole_domain[0]:
    center = gmsh.model.occ.getCenterOfMass(domain[0], domain[1])
    mass = gmsh.model.occ.getMass(domain[0], domain[1])

    
    # Identify the background circle by its center of mass
    if np.allclose(center, [0, 0, 0]):
        background_surfaces.append(domain[1])


    elif np.isclose(center[0], -1):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], inner_tag)
        inner_tag +=1
        other_surfaces.append(domain)


    elif np.isclose(center[0], +1):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], outer_tag)
        outer_tag +=1
        other_surfaces.append(domain)


    elif np.isclose(center[0], -2):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], inner_tag)
        outer_tag +=1
        other_surfaces.append(domain)


    elif np.isclose(center[0], 2):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], outer_tag)
        inner_tag +=1
        other_surfaces.append(domain)
        




# Add marker for the vacuum.
gmsh.model.addPhysicalGroup(2, background_surfaces, tag=0)

# Create mesh resolution that is fine around the wires and
# make the grid coarse further away from the ring.
gmsh.model.mesh.field.add("Distance", 1)
edges = gmsh.model.getBoundary(other_surfaces, oriented=False)
gmsh.model.mesh.field.setNumbers(1, "EdgesList", [e[1] for e in edges])
gmsh.model.mesh.field.add("Threshold", 2)
gmsh.model.mesh.field.setNumber(2, "IField", 1)
gmsh.model.mesh.field.setNumber(2, "LcMin", r_wire / 2)
gmsh.model.mesh.field.setNumber(2, "LcMax", 5 * r_wire)
gmsh.model.mesh.field.setNumber(2, "DistMin", 2 * r_wire)
gmsh.model.mesh.field.setNumber(2, "DistMax", 4 * r_wire)
gmsh.model.mesh.field.setAsBackgroundMesh(2)
gmsh.option.setNumber("Mesh.Algorithm", 7)

# Create a mesh for this system.
gmsh.model.mesh.generate(dim)

# Bring the mesh into FEniCSx.
mesh, subdomains, boundaries = dolfinx.io.gmshio.model_to_mesh(
    gmsh.model, comm=mpi4py.MPI.COMM_WORLD, rank=0, gdim=2)

gmsh.finalize()

# This loop will assign material properties to each cell in our model.
# In this case, it is the relative magnetic permeability and current density.

# Define a simple function space for properties.
Q = dolfinx.fem.FunctionSpace(mesh, ("DG", 0))

# Get the list of materials.
material_tags = np.unique(subdomains.values)

# Define functions for current density and magnetic permeability.
mu = dolfinx.fem.Function(Q)
J = dolfinx.fem.Function(Q)

# Only some regions carry current. Initialize all current densities to zero.
J.x.array[:] = 0.0

# Now, cycle over all objects and assign material properties. 
for tag in material_tags:
    cells = subdomains.find(tag)
    
    # Set values for magnetic permeability.
    if tag == 0:
        # Vacuum
        mu_ = mu_background
    elif tag == 1:
        # Ring
        mu_ = mu_ring
    else:
        # Wire
        mu_ = mu_wire

    mu.x.array[cells] = np.full_like(cells, mu_, dtype=petsc4py.PETSc.ScalarType)
    
    # Set nonzero current densities.
    if tag in range(2, 2+N):
        J.x.array[cells] = np.full_like(cells, J0, dtype=petsc4py.PETSc.ScalarType)
    elif tag in range(2+N, 2*N + 2):
        J.x.array[cells] = np.full_like(cells, -J0, dtype=petsc4py.PETSc.ScalarType)
        
## Set up the finite element problem.

# Define trial and test functions.
V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 2))

# Define the trial and test functions.
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Create a function to store the solution.
# This is the vector potential.  A_x = A_y = 0.
A_z = dolfinx.fem.Function(V)

# Identify the domain and boundary.
D = mesh.topology.dim
Omega = dolfinx.mesh.locate_entities_boundary(mesh, D-1, lambda x: np.full(x.shape[1], True))
dOmega = dolfinx.fem.locate_dofs_topological(V, D-1, Omega)

# Force the potential to vanish on the boundary.
bc = dolfinx.fem.dirichletbc(petsc4py.PETSc.ScalarType(0), dOmega, V)

# Define the Poisson equation we are trying to solve.
a = (1 / mu) * ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = 4 * ufl.pi * J * v * ufl.dx

# Define the problem.
problem = dolfinx.fem.petsc.LinearProblem(a, L, u=A_z, bcs=[bc])

# Solve the problem.
problem.solve()


# Compute the magnetic field.
W = dolfinx.fem.VectorFunctionSpace(mesh, ("CG", 2))
B = dolfinx.fem.Function(W)
B_expr = dolfinx.fem.Expression(ufl.as_vector((A_z.dx(1), -A_z.dx(0))), W.element.interpolation_points())
B.interpolate(B_expr)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 20%] Meshing curve 2 (Ellipse)
Info    : [ 30%] Meshing curve 3 (Ellipse)
Info    : [ 40%] Meshing curve 4 (Ellipse)
Info    : [ 50%] Meshing curve 5 (Ellipse)
Info    : [ 60%] Meshing curve 6 (Ellipse)
Info    : [ 70%] Meshing curve 7 (Ellipse)
Info    : [ 80%] Meshing curve 8 (Ellipse)
Info    : [ 90%] Meshing curve 9 (Ellipse)
Info    : Done meshing 1D (Wall 0.00250608s, CPU 0.003246s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 2 (Plane, Bamg)
Info    : [  0%] BAMG succeeded 22 vertices 29 triangles
Info    : [  0%] BAMG succeeded 22 vertices 29 triangles
Info    : [ 20%] Meshing surface 3 (Plane, Bamg)
Info    : [ 20%] BAMG succeeded 22 vertices 29 triangles
Info    : [ 20%] BAMG succeeded 22 vertices 29 triangles
Info    : [ 30%] Meshing surface 4 (Plane, Bamg)
Info    : [ 30%] BAMG succeeded 21 vertices 27 triangles
Info    : [ 30%] BAMG succeeded 21 vertices 27 triangles
Info    : [ 40%]

In [77]:
multiphenicsx.io.plot_scalar_field(A_z,"Vector Potential", warp_factor=1)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [61]:
multiphenicsx.io.plot_vector_field(B,"Magnetic Field", glyph_factor=0.1)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)